# Coleccionando data de Covidtracking

Este notebook nos permitirá recolectar la información de la siguiente https://covidtracking.com/data/api y guardarla en una base de datos Postgresql

In [1]:
import pandas as pd
import numpy as np
import psycopg2

In [2]:
URL_COVID_TRACKING = 'https://covidtracking.com/api/v1/states/daily.csv'
HOST = '127.0.0.1'
DATABASE = 'covid_tracking'
USER = 'postgres'
PASSWORD = 'reyson07'

In [3]:
def getDataApiCovidTracking(url):
    dfCasesStates = pd.read_csv(URL_COVID_TRACKING,
                                 usecols=['date', 'state', 'positive','totalTestResults'],
                                 parse_dates=['date'],
                                 index_col=['state', 'date'],
                                 squeeze=True).sort_index()
    return dfCasesStates

In [4]:
def get_connection():
    return psycopg2.connect(host=HOST,
                            database=DATABASE,
                            user=USER,
                            password=PASSWORD)

In [5]:
def insert_covid_time(dfCasesStates):
    dates = dfCasesStates.index.get_level_values("date").unique()
    for i in range(dates.shape[0]):
        try:
            conn = get_connection()
            cur = conn.cursor()
            call_sp = f"CALL public.insert_covid_time('{dates[i]}');"
            cur.execute(call_sp)
            conn.commit()
        except(Exception,psycopg2.DatabaseError) as error:
            print(f'ERROR: {error}')
            print(call_sp)
        finally:
            cur.close()

In [6]:
def insert_covid_state(dfCasesStates):
    states = dfCasesStates.index.get_level_values("state").unique()
    for i in range(states.shape[0]):
        try:
            conn = get_connection()
            cur = conn.cursor()
            call_sp = f"CALL public.insert_covid_states('{states[i]}');"
            cur.execute(call_sp)
            conn.commit()
        except(Exception,psycopg2.DatabaseError) as error:
            print(f'ERROR: {error}')
            print(call_sp)
        finally:
            cur.close()

In [7]:
def insert_positive_cases(dfCasesStates):
    states = dfCasesStates.index.get_level_values("state")
    dates = dfCasesStates.index.get_level_values("date")
    dfCasesStates = dfCasesStates.fillna(-1)
    for i in range(dfCasesStates.shape[0]):
        try:
            conn = get_connection()
            cur = conn.cursor()
            call_sp = f"CALL public.insert_covid_pos_cases('{states[i]}','{dates[i]}',{int(dfCasesStates.values[i][0])});"
            cur.execute(call_sp)
            conn.commit()
        except(Exception,psycopg2.DatabaseError) as error:
            print(f'ERROR: {error}')
            print(call_sp)
        finally:
            cur.close()

In [8]:
def insert_test_results(dfCasesStates):
    states = dfCasesStates.index.get_level_values("state")
    dates = dfCasesStates.index.get_level_values("date")
    dfCasesStates = dfCasesStates.fillna(-1)
    for i in range(dfCasesStates.shape[0]):
        try:
            conn = get_connection()
            cur = conn.cursor()
            call_sp = f"CALL public.insert_covid_test_results('{states[i]}','{dates[i]}',{int(dfCasesStates.values[i][1])});"
            cur.execute(call_sp)
            conn.commit()
        except(Exception,psycopg2.DatabaseError) as error:
            print(f'ERROR: {error}')
            print(call_sp)
        finally:
            cur.close()

In [9]:
dfDataApiTracking = getDataApiCovidTracking(URL_COVID_TRACKING)
insert_covid_time(dfDataApiTracking)
insert_covid_state(dfDataApiTracking)
insert_positive_cases(dfDataApiTracking)

In [12]:
insert_test_results(dfDataApiTracking)